In [ ]:
from whar_datasets.adapters.torch_adapter import TorchAdapter
from whar_datasets.support.getter import WHARDatasetID, get_dataset_cfg

In [ ]:
# initialize dataset config given a dataset id
cfg = get_dataset_cfg(dataset_id=WHARDatasetID.WISDM)

# initialize dataset and preprocess
dataset = TorchAdapter(cfg=cfg)
dataset.preprocess()

# postprocess for a given split and get dataloaders
dataset.postprocess(split_group_index=0)
dataloaders = dataset.get_dataloaders(batch_size=32)